In [8]:
import pandas, seaborn, scipy, numpy, matplotlib, collections, itertools, math, functools, sys, sklearn, os

In [44]:
ega_filenames = pandas.read_table("../data/ega_contents.aspera.tsv", header=None)[0]
ega_filenames = pandas.Series([x.replace(".aes", "") for x in ega_filenames])

downloaded_file_paths = pandas.read_table("../data/downloaded_file_paths.txt", header=None)[0]
#downloaded_filenames = set(os.path.basename(x) for x in downloaded_file_paths)
#downloaded_filenames = pandas.Series([x.strip("_") for x in downloaded_filenames])

ISSUE_EGA_FILENAMES = set()
def lookup_filename(substring):
    results = downloaded_file_paths[downloaded_file_paths.str.contains(substring)]
    ega_results = set(ega_filenames[ega_filenames.str.contains(substring)])

    if len(results) == 0:
        print("NOT_FOUND: %s, EGA results: %s" % (substring, ega_results))
        ISSUE_EGA_FILENAMES.update(ega_results)
        return "NOT_FOUND:%s" % substring
    if len(results) > 1:
        print("MULTIPLE_FOUND: %s, EGA results: %s" % (substring, ega_results))
        #ISSUE_EGA_FILENAMES.update(ega_results)
        return "MULTIPLE_FOUND:%s" % substring
    return list(results)[0]


In [27]:
#missing = [x for x in ega_filenames if x not in downloaded_filenames]
#print("Missing files (%d): %s" % (len(missing), missing))


Missing files (265): ['EGAZ00001018569_2014_ICGC_130906_D81P8DQ1_0157_C270DACXX.nopd.AOCS_079_ICGCDBDE20130916014.rsem.bam', 'EGAZ00001018570_2014_ICGC_IcgcOvarian_AOCS162_1DNA_1NormalBlood_ICGCDBDE20130904006_IlluminaIGNOutsourcing_NoCapture_Bwa_HiSeq.jpearson.bam', 'EGAZ00001018571_2014_ICGC_130906_D81P8DQ1_0153_C2704ACXX.nopd.AOCS_094_ICGCDBDE20130916020.rsem.bam', 'EGAZ00001018572_2014_ICGC_130906_D81P8DQ1_0158_D2B80ACXX.nopd.AOCS_085_ICGCDBDE20130916018.rsem.bam', 'EGAZ00001018573_2014_ICGC_IcgcOvarian_AOCS096_1DNA_1NormalBlood_ICGCDBPC20130205071_IlluminaIGNOutsourcing_NoCapture_Bwa_HiSeq.jpearson.bam', 'EGAZ00001018574_2014_ICGC_131206_EXTERN_0044_AC2JWTACXX.nopd.AOCS_116_ICGCDBDE20131122018.rsem.bam', 'EGAZ00001018575_2014_ICGC_IcgcOvarian_AOCS161_1DNA_7PrimaryTumour_ICGCDBDE20130904015_IlluminaIGNOutsourcing_NoCapture_Bwa_HiSeq.jpearson.bam', 'EGAZ00001018576_2014_ICGC_IcgcOvarian_AOCS143_1DNA_7PrimaryTumour_ICGCDBPC20130205158_IlluminaIGNOutsourcing_NoCapture_Bwa_HiSeq.jpears

In [45]:
samples = pandas.read_table("../data/sample.tsv")
specimens = pandas.read_table("../data/specimen.tsv")
ids = pandas.read_csv("../data/ICGC_IDs_19Oct2015_External_modified.csv")

sources = pandas.merge(ids, samples, left_on='DNA_biospecimen', right_on="submitted_sample_id", how='inner')
sources = pandas.merge(sources, specimens, on="icgc_specimen_id", how='inner').copy()


assert all(sources.submitted_donor_id_x == sources.submitted_donor_id_y)
assert all(sources.icgc_donor_id_x == sources.icgc_donor_id_y)

sources["cohort"] = "AOCS"
sources["donor"] = sources.submitted_donor_id_x
sources["tissue_type"] = sources.SpecimenType.map({"Tumour": "solid", "Ascites": "ascites"})
sources["timepoint"] = sources.CollectionPoint.map(
    {"Primary": "primary", "Recurrence": "recurrence", "Autopsy": "recurrence"})

sources["source_id"] = sources.DNA_biospecimen
sources.index = sources.source_id

sources["treated"] = sources.specimen_donor_treatment_type == "other therapy"
sources["metastasis"] = sources.specimen_type_description.str.contains("metastasis")
sources["interval_days"] = sources.specimen_interval

print('looking for dna')
sources["bam_path_tumor_dna"] = [lookup_filename(row.DNA_sample_string) for (i,row) in sources.iterrows()]

print("looking for rna")
sources["bam_path_tumor_rna"] = [lookup_filename(row["RNA data file"]) for (i,row) in sources.iterrows()]

sources.loc["AOCS-170-1-8", "treated"] = False  # weird incorrect data point based on email from Elizabeth Christie


assert sources["source_id"].nunique() == 114
assert len(sources["source_id"]) == 114

simple_sources = sources[
    "source_id,donor,cohort,treated,timepoint,metastasis,tissue_type,interval_days,bam_path_tumor_dna,bam_path_tumor_rna".split(",")
]

sources.to_csv("../data/sources.full.csv", index=False)
simple_sources.to_csv("../data/sources.simple.csv", index=False)


looking for dna
NOT_FOUND: ICGCDBPC20130205002, EGA results: {'EGAZ00001018601_2014_ICGC_IcgcOvarian_AOCS001_1DNA_7PrimaryTumour_ICGCDBPC20130205002_IlluminaIGNOutsourcing_NoCapture_Bwa_HiSeq.jpearson.bam'}
MULTIPLE_FOUND: ICGCDBPC20130205009, EGA results: {'EGAZ00001018597_2014_ICGC_IcgcOvarian_AOCS034_1DNA_7PrimaryTumour_ICGCDBPC20130205009_IlluminaIGNOutsourcing_NoCapture_Bwa_HiSeq.jpearson.bam'}
MULTIPLE_FOUND: ICGCDBPC20130205007, EGA results: {'EGAZ00001018766_2014_ICGC_IcgcOvarian_AOCS034_1DNA_12Ascites_ICGCDBPC20130205007_IlluminaIGNOutsourcing_NoCapture_Bwa_HiSeq.jpearson.bam'}
NOT_FOUND: ICGCDBPC20130205011, EGA results: {'EGAZ00001018609_2014_ICGC_IcgcOvarian_AOCS055_1DNA_7PrimaryTumour_ICGCDBPC20130205011_IlluminaIGNOutsourcing_NoCapture_Bwa_HiSeq.jpearson.bam'}
NOT_FOUND: ICGCDBPC20130205019, EGA results: {'EGAZ00001018614_2014_ICGC_IcgcOvarian_AOCS059_1DNA_7PrimaryTumour_ICGCDBPC20130205019_IlluminaIGNOutsourcing_NoCapture_Bwa_HiSeq.jpearson.bam'}
NOT_FOUND: ICGCDBPC20130

In [46]:
print("\n".join(["EGAD00001000877/%s.aes" % x for x in ISSUE_EGA_FILENAMES]))

EGAD00001000877/EGAZ00001018609_2014_ICGC_IcgcOvarian_AOCS055_1DNA_7PrimaryTumour_ICGCDBPC20130205011_IlluminaIGNOutsourcing_NoCapture_Bwa_HiSeq.jpearson.bam.aes
EGAD00001000877/EGAZ00001018614_2014_ICGC_IcgcOvarian_AOCS059_1DNA_7PrimaryTumour_ICGCDBPC20130205019_IlluminaIGNOutsourcing_NoCapture_Bwa_HiSeq.jpearson.bam.aes
EGAD00001000877/EGAZ00001018681_2014_ICGC_IcgcOvarian_AOCS147_1DNA_7PrimaryTumour_ICGCDBPC20130205166_IlluminaIGNOutsourcing_NoCapture_Bwa_HiSeq.jpearson.bam.aes
EGAD00001000877/EGAZ00001018828_2014_ICGC_IcgcOvarian_AOCS149_1DNA_7PrimaryTumour_ICGCDBPC20130205170_IlluminaIGNOutsourcing_NoCapture_Bwa_HiSeq.jpearson.bam.aes
EGAD00001000877/EGAZ00001018651_2014_ICGC_IcgcOvarian_AOCS167_1DNA_16TumourMetastasisToDistantLocation_ICGCDBDE20130904027_IlluminaIGNOutsourcing_NoCapture_Bwa_HiSeq.jpearson.bam.aes
EGAD00001000877/EGAZ00001018588_2014_ICGC_IcgcOvarian_AOCS122_1DNA_7PrimaryTumour_ICGCDBPC20130205102_IlluminaIGNOutsourcing_NoCapture_Bwa_HiSeq.jpearson.bam.aes
EGAD000

In [ ]:
original_sources = pandas.read_csv("../data/sources.aocs_only.csv")
original_sources
original_sources.treated.value_counts()

In [ ]:
original_sources.donor.nunique()
simple_sources.donor.nunique()

In [ ]:
original_sources.shape, simple_sources.shape

In [ ]:
simple_sources.treated.value_counts(), original_sources.treated.value_counts()

In [ ]:
simple_sources.metastasis.value_counts(), original_sources.metastasis.value_counts()